# Movie Recommendation System

In this notebook we use the trained collaborative filtering model to generate personalized movie recommendations.

In [ ]:
import tensorflow as tf
import pandas as pd

## Step 1: Load and preprocess the dataset

We load the preprocessed dataset and ensure that both `userId` and `movieId` are strings.  
This is required for consistency with the trained model.

In [ ]:
ratings_df = pd.read_csv("ratings_meta_small.csv")
ratings_df["userId"] = ratings_df["userId"].astype(str)
ratings_df["movieId"] = ratings_df["movieId"].astype(str)

 ## Step 2: Load the trained model

We load the saved Keras model that was trained in the previous notebook.

In [108]:
model = tf.keras.models.load_model("recommender_model2.keras")

## Step 3: Define the recommendation function

The function `recommend_movies` does the following:

1. Converts the `userId` to string for consistency.  
2. Collects all movies the user has already rated (`seen`).  
3. Builds a candidate set of unique movies the user has **not** rated yet.  
4. Creates TensorFlow string tensors for user- and movieIDs.  
5. Uses the trained model to predict ratings for all candidate movies.  
6. Adds predictions as a new column in the candidates DataFrame.  
7. Sorts by predicted rating and selects the top-N recommendations.  
8. Returns only the relevant columns (`movieId`, `title`, `pred_rating`).  

This function generates personalized recommendations by filtering out already-seen movies and ranking unseen ones.

In [109]:
def recommend_movies(model, userId, ratings_df, top_n=5):
    userId = str(userId)


    # Movies already rated by the user
    seen = ratings_df[ratings_df["userId"].astype(str) == userId]["movieId"].astype(str).tolist()

    # Unique candidate movies
    candidates = ratings_df.drop_duplicates(subset="movieId")
    candidates = candidates[~candidates["movieId"].astype(str).isin(seen)]

    # Inputs (TensorFlow tensors with dtype=tf.string)
    movieIds = tf.constant(candidates["movieId"].astype(str).tolist(), dtype=tf.string)
    userIds = tf.constant([userId] * len(movieIds), dtype=tf.string)

    # Predictions
    preds = model.predict({"userId": userIds, "movieId": movieIds}, verbose=0).flatten()

    # Reset index to keep iloc aligned with preds
    candidates = candidates.reset_index(drop=True)

    # Add predictions as a new column
    candidates["pred_rating"] = preds

    # Sort descending by predicted rating
    recommended = candidates.sort_values("pred_rating", ascending=False).head(top_n)

    # Keep only the relevant columns
    recommended = recommended[["movieId", "title", "pred_rating"]]

    return recommended

## Step 4: Test the recommender system on a specific user

We generate the top-5 recommendations for `userId=1`.

In [116]:
recs = recommend_movies(model, userId="1", ratings_df=ratings_df, top_n=5)
recs = recs.reset_index(drop=True)
recs.index = recs.index + 1
print(recs[['title', 'pred_rating']])


                 title  pred_rating
1              Yojimbo     3.765936
2        The Godfather     3.746414
3           Fight Club     3.711394
4  A Face in the Crowd     3.709913
5  Hachi: A Dog's Tale     3.683201


## Step 5: Generate recommendations for random users

We test the recommender system on several randomly selected users to see how recommendations differ between them.

In [111]:
import random

unique_users = ratings_df["userId"].unique()

for x in range(5):
    uid = random.choice(unique_users)
    print(f"\n===== User {uid} =====")
    print(recommend_movies(model, userId=uid, ratings_df=ratings_df, top_n=5))


===== User 272 =====
     movieId                 title  pred_rating
6460    4696  Zorro, The Gay Blade     4.495519
4294    3739   Trouble in Paradise     4.480273
1986    7116            Diabolique     4.478728
2368    6669                 Ikiru     4.455186
2962     838                  Emma     4.452141

===== User 548 =====
     movieId                                                       title  \
869     3030                                                     Yojimbo   
6681    4783  The Endurance: Shackleton's Legendary Antarctic Expedition   
6361    4696                                        Zorro, The Gay Blade   
1910    7116                                                  Diabolique   
1696  109374                                    The Grand Budapest Hotel   

      pred_rating  
869      4.314000  
6681     4.293063  
6361     4.284800  
1910     4.258125  
1696     4.248127  

===== User 136 =====
     movieId                 title  pred_rating
2957     838         